# Evaluate RAG with LlamaIndex

In this notebook we will look into building an RAG pipeline and evaluating it with LlamaIndex. It has following 3 sections.

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG with LlamaIndex.
3. Evaluating RAG with LlamaIndex.

## Build RAG system.

Now that we have understood the significance of RAG system, let's build a simple RAG pipeline.

In [ ]:
!pip install pyarrow
!pip install llama-index
!pip install ragas
!pip install datasets

!pip install llama-index-llms-langchain
!pip install langchainhub
!pip install llama-index-llms-fireworks

!pip install llama-index-llms-anyscale
!pip install llama-index-embeddings-anyscale

!pip install anyscale
!pip install openai
!pip install llama-index-embeddings-huggingface

In [ ]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio
nest_asyncio.apply()

#from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI
#from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

from langchain import hub
from llama_index.core.prompts import LangchainPromptTemplate

from llama_index.llms.anyscale import Anyscale
from llama_index.embeddings.anyscale import AnyscaleEmbedding

import os
import pandas as pd

from datasets import Dataset

Set Your OpenAI API Key

#### Load Data and Build Index.

In [ ]:
documents = SimpleDirectoryReader("/content/Data").load_data()

# Define an LLM
#eva_llm = OpenAI(model="gpt-3.5")

# Build index with a chunk_size of 512
# node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
# nodes = node_parser.get_nodes_from_documents(documents)
# vector_index = VectorStoreIndex(nodes)

#service_context = ServiceContext.from_defaults(llm=Anyscale(model="mistralai/Mixtral-8x7B-Instruct-v0.1"),
                                               #embed_model=OpenAIEmbedding(model="text-embedding-ada-002"))

#index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Convert the index into a query engine
#query_engine = index.as_query_engine()
#response_vector = query_engine.query("Trường Đại học Khoa học tự nhiên, ĐHQG-HCM đào tạo ngành nào nhiều sinh viên nhất?")
#response_vector.response

In [ ]:
#from llama_index.core import Document

#document = [Document(text="\n\n".join([doc.text for doc in documents]))]

In [ ]:
os.environ['ANYSCALE_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''
os.environ['HUGGINGFACE_API_KEY']=''

In [ ]:
service_context = ServiceContext.from_defaults(llm=Anyscale(model="mistralai/Mixtral-8x7B-Instruct-v0.1"),
                                               embed_model="local:BAAI/bge-small-en-v1.5")

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

<ipython-input-6-643ba364ee1c>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=Anyscale(model="mistralai/Mixtral-8x7B-Instruct-v0.1"),
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in ve

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)
# use this for testing
#vector_retriever = index.as_retriever(similarity_top_k=5)

In [ ]:
response_vector = query_engine.query("Học phí ngành Truyền thông đa phương tiện chương trình tiêu chuẩn trường Đại học Khoa học Xã hội và Nhân văn, ĐHQG-HCM?")
response_vector.response

' The tuition fee for the major of Multimedia Communication (chương trình tiêu chuẩn) at the School of Social Sciences and Humanities, National University of Ho Chi Minh City is not explicitly stated in the provided context. However, it is mentioned that the fee for the Department of Journalism and Communication, which includes the Multimedia Communication major, is around 26.4 million VND per year. It is recommended to contact the school directly for the most accurate and up-to-date information.'

In [ ]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
# to do this, you need to use the langchain object

from langchain import hub

langchain_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from llama_index.core.prompts import LangchainPromptTemplate

lc_prompt_tmpl = LangchainPromptTemplate(
    template=langchain_prompt,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": lc_prompt_tmpl}
)

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
# write prompt template with functions
qa_prompt_tmpl_str = """\
Trả lời câu hỏi sau về thông tin tuyển sinh của các trường Đại học Quốc Gia TP Hồ Chí Minh dựa trên thông tin ngữ cảnh đã cung cấp sau đây
Thông tin ngữ cảnh dưới đây.
---------------------
{context_str}
---------------------

Query: {query_str}.
Trả lời: \
"""

def get_context():
    # Replace this with your actual logic to retrieve relevant context from documents
    relevant_docs = [doc for doc in documents]
    return "\n".join([doc.text for doc in relevant_docs])

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str
)

In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Trả lời câu hỏi sau về thông tin tuyển sinh của các trường Đại học Quốc Gia TP Hồ Chí Minh dựa trên thông tin ngữ cảnh đã cung cấp sau đây
Thông tin ngữ cảnh dưới đây.
---------------------
{context_str}
---------------------

Query: {query_str}.
Trả lời: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

Build a QueryEngine and start querying.

In [ ]:
"""index = build_sentence_window_index(
    document,
    llm=OpenAI(model = "gpt-3.5-turbo", temperature = 0.1,
               prompt="Bạn là chatbot tư vấn tuyển sinh của các trường Đại học thành viên Đại học Quốc gia TP.HCM. Bạn được tạo ra bởi đội ngũ chuyên gia giáo dục với mục đích giúp học sinh tìm hiểu thông tin và đưa ra quyết định sáng suốt trong việc lựa chọn ngành học và trường đại học phù hợp. Bạn có thể cung cấp thông tin chi tiết về các trường đại học thành viên, ngành học, chương trình đào tạo, học phí, học bổng, v.v. Hãy sử dụng tiếng Việt là ngôn ngữ mặc định."),
    save_dir="./sentence_index",
)"""

'index = build_sentence_window_index(\n    document,\n    llm=OpenAI(model = "gpt-3.5-turbo", temperature = 0.1,\n               prompt="Bạn là chatbot tư vấn tuyển sinh của các trường Đại học thành viên Đại học Quốc gia TP.HCM. Bạn được tạo ra bởi đội ngũ chuyên gia giáo dục với mục đích giúp học sinh tìm hiểu thông tin và đưa ra quyết định sáng suốt trong việc lựa chọn ngành học và trường đại học phù hợp. Bạn có thể cung cấp thông tin chi tiết về các trường đại học thành viên, ngành học, chương trình đào tạo, học phí, học bổng, v.v. Hãy sử dụng tiếng Việt là ngôn ngữ mặc định."),\n    save_dir="./sentence_index",\n)'

In [ ]:
# hyde = HyDEQueryTransform(include_original=True)
# query_engine = TransformQueryEngine(query_engine, hyde)

Check response.

In [ ]:
import pandas as pd

# Specify the file path
file_path = r'/content/prompt.xlsx'

# Read the Excel file into a DataFrame
test = pd.read_excel(file_path)

# Print the DataFrame
print(test)

                                            question  \
0  Học phí ngành Hoá học Chương trình Chất lượng ...   
1  Ngành Vật lý học Chương trình đào tạo tăng cườ...   
2  Thời gian đào tạo ngành Y học cổ truyền của kh...   
3  Ngành Kinh tế quốc tế của trường Đại học An Gi...   
4  Trường Đại học An Giang, ĐHQG-HCM có đào tạo n...   
5  Học phí ngành Truyền thông đa phương tiện chươ...   
6  Trong Đại học Quốc gia TP.HCM, liệt kê các trư...   

                                        ground_truth  
0                            ['40.000.000 đồng/năm']  
1                                   ['50 sinh viên']  
2                                          ['8 năm']  
3                                   ['80 sinh viên']  
4                                             ['Có']  
5                        ['22.000.000 đồng/năm học']  
6  ['Trường Đại học Kinh tế - Luật, ĐHQG-HCM; trư...  


We have built a RAG pipeline and now need to evaluate its performance. We can assess our RAG system/query engine using LlamaIndex's core evaluation modules. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

In [ ]:
questions = test["question"].to_list()

In [ ]:
answers = []
contexts = []

for i in questions:
  response_vector = query_engine.query(i)
  answers.append(response_vector)
  contexts.append([a.get_text() for a in response_vector.source_nodes])

In [ ]:
ground_truths = [[a] for a in test["ground_truth"]]

In [ ]:
for num, i in enumerate(ground_truths):
    if type(i) != str:
        ground_truths[num] = str(i)

In [ ]:
answers = [str(i) for i in answers]

In [ ]:
datasample = {
    "question": questions,
    "contexts": contexts,
    "answer": answers,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(datasample)

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
]

In [ ]:
from ragas import evaluate

result = evaluate(dataset=dataset, metrics=metrics, is_async=True, raise_exceptions=False)

print(result)

In [ ]:
rs = result.to_pandas()
rs

In [ ]:
rs.to_excel("testset1_any5_prm_1.xlsx")